# MLP

multi_layer_perceptron

In [1]:
import numpy as np

# ml
from sklearn.model_selection import (
    KFold,
    cross_validate,
    GridSearchCV,
    RepeatedKFold,
    train_test_split,
)
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from utils.metrics import calculate_metrics, get_ccp_scoring, print_results_table
from utils.datasets import load_and_split_data

In [2]:
data_path = "../../data/processed/rdc_data_cleaned.csv"
X_raw, Y_raw = load_and_split_data(data_path, test_size=0.1, is_split=False)


scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X_raw)
y_scaled = scaler_y.fit_transform(Y_raw)

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.1, random_state=42
)

# 打印划分后的数据集大小
print("训练集大小:", len(x_train))
print("测试集大小:", len(x_test))

训练集大小: 350
测试集大小: 39


In [13]:
model_name = "single-layer MLP"

x_train, x_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.1, random_state=42
)

grid_search = GridSearchCV(
    MLPRegressor(activation="relu", max_iter=2000, random_state=42),
    param_grid=[
        {
            "hidden_layer_sizes": range(1500, 1700, 5),
            "solver": ["sgd", "adam"],
            "learning_rate": ["constant", "invscaling", "adaptive"],
            "learning_rate_init": [0.001],
        },
    ],
    scoring="neg_mean_absolute_error",
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    verbose=1,
    n_jobs=-1,
)
grid_search.fit(x_train, y_train)
calculate_metrics(
    scaler_y.inverse_transform(grid_search.best_estimator_.predict(x_test)),
    scaler_y.inverse_transform(y_test),
    print_metrics=True,
    title=model_name,
)
print("Best Parameters for", model_name, ":", grid_search.best_params_)
print("Best Score for", model_name, ":", grid_search.best_score_)
print("=" * 47 + "\n" * 2)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


            single-layer MLP             
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE  │ RMSE(Pa) ┃
┠─────────┼─────────┼────────┼──────────┨
┃ Airflow │  59.79  │ 5.87%  │  87.06   ┃
┃ Pres    │  3.30   │ 51.61% │   5.08   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━┷━━━━━━━━━━┛

Best Parameters for single-layer MLP : {'hidden_layer_sizes': 1600, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'solver': 'adam'}
Best Score for single-layer MLP : -0.1305227207528842


